<a href="https://colab.research.google.com/github/jeibloo/DS-Unit-2-Regression-Classification/blob/master/module4/assignment_regression_classification_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 4

## Assignment

- [X] Watch Aaron Gallant's [video #1](https://www.youtube.com/watch?v=pREaWFli-5I) (12 minutes) & [video #2](https://www.youtube.com/watch?v=bDQgVt4hFgY) (9 minutes) to learn about the mathematics of Logistic Regression.
- [X] Do train/validate/test split with the Tanzania Waterpumps data.
- [X] Do one-hot encoding. (Remember it may not work with high cardinality categoricals.)
- [X] Use scikit-learn for logistic regression.
- [X] Get your validation accuracy score.
- [ ] Get and plot your coefficients.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

> [Do Not Copy-Paste.](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit) You must type each of these exercises in, manually. If you copy and paste, you might as well not even do them. The point of these exercises is to train your hands, your brain, and your mind in how to read, write, and see code. If you copy-paste, you are cheating yourself out of the effectiveness of the lessons.


## Stretch Goals

### Doing
- [ ] Add your own stretch goal(s) !
- [ ] Clean the data. For ideas, refer to [The Quartz guide to bad data](https://github.com/Quartz/bad-data-guide),  a "reference to problems seen in real-world data along with suggestions on how to resolve them." One of the issues is ["Zeros replace missing values."](https://github.com/Quartz/bad-data-guide#zeros-replace-missing-values)
- [ ] Make exploratory visualizations.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).


#### Exploratory visualizations

Visualize the relationships between feature(s) and target. I recommend you do this with your training set, after splitting your data. 

For this problem, you may want to create a new column to represent the target as a number, 0 or 1. For example:

```python
train['functional'] = (train['status_group']=='functional').astype(int)
```



You can try [Seaborn "Categorical estimate" plots](https://seaborn.pydata.org/tutorial/categorical.html) for features with reasonably few unique values. (With too many unique values, the plot is unreadable.)

- Categorical features. (If there are too many unique values, you can replace less frequent values with "OTHER.")
- Numeric features. (If there are too many unique values, you can [bin with pandas cut / qcut functions](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html?highlight=qcut#discretization-and-quantiling).)

You can try [Seaborn linear model plots](https://seaborn.pydata.org/tutorial/regression.html) with numeric features. For this problem, you may want to use the parameter `logistic=True`

You do _not_ need to use Seaborn, but it's nice because it includes confidence intervals to visualize uncertainty.

#### High-cardinality categoricals

This code from the previous assignment demonstrates how to replace less frequent values with 'OTHER'

```python
# Reduce cardinality for NEIGHBORHOOD feature ...

# Get a list of the top 10 neighborhoods
top10 = train['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10,
# replace the neighborhood with 'OTHER'
train.loc[~train['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
test.loc[~test['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
```

#### Pipelines

[Scikit-Learn User Guide](https://scikit-learn.org/stable/modules/compose.html) explains why pipelines are useful, and demonstrates how to use them:

> Pipeline can be used to chain multiple estimators into one. This is useful as there is often a fixed sequence of steps in processing the data, for example feature selection, normalization and classification. Pipeline serves multiple purposes here:
> - **Convenience and encapsulation.** You only have to call fit and predict once on your data to fit a whole sequence of estimators.
> - **Joint parameter selection.** You can grid search over parameters of all estimators in the pipeline at once.
> - **Safety.** Pipelines help avoid leaking statistics from your test data into the trained model in cross-validation, by ensuring that the same samples are used to train the transformers and predictors.

### Reading
- [ ] [How (and why) to create a good validation set](https://www.fast.ai/2017/11/13/validation-sets/)
- [ ] [Always start with a stupid model, no exceptions](https://blog.insightdatascience.com/always-start-with-a-stupid-model-no-exceptions-3a22314b9aaa)
- [ ] [Statistical Modeling: The Two Cultures](https://projecteuclid.org/download/pdf_1/euclid.ss/1009213726)
- [ ] [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way (without an excessive amount of formulas or academic pre-requisites).



In [556]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # category_encoders, version >= 2.0
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade category_encoders pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module4')

Requirement already up-to-date: category_encoders in /usr/local/lib/python3.6/dist-packages (2.0.0)
Requirement already up-to-date: pandas-profiling in /usr/local/lib/python3.6/dist-packages (2.3.0)
Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.1.0)
Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd

train_features = pd.read_csv('../data/tanzania/train_features.csv')
train_labels = pd.read_csv('../data/tanzania/train_labels.csv')
test_features = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

assert train_features.shape == (59400, 40)
assert train_labels.shape == (59400, 2)
assert test_features.shape == (14358, 40)
assert sample_submission.shape == (14358, 2)

# SPLITPOINT: Train/Validate/Test

In [0]:
### Well, train and test are already separate. I'll split train in two.
# We're going to go randomly in the training set
from sklearn.model_selection import train_test_split

# Here's what we're training with
X_train = train_features
# Here's what we're training against
y_train = train_labels['status_group']
# Encode y_train: 'functional', 'functional needs repair', 'non functional'
#y_train = y_train.replace({'functional':2,
#                           'functional needs repair':1,
#                           'non functional':0})

In [560]:
### This sthtuff shouldn't probably be a thing maybe.
# Need to cut y_train down to what the size of this is.
test = pd.merge(X_train,y_train, left_index=True, right_index=True)
X_train = X_train[(X_train['population'] >= np.percentile(X_train['population'],0.25))&
                  (X_train['population'] <= np.percentile(X_train['population'],99.75))]
test = test[(test['population'] >= np.percentile(test['population'],0.25))&
                  (test['population'] <= np.percentile(test['population'],99.75))]
y_train = test['status_group']
print(X_train.shape,y_train.shape)

(59251, 40) (59251,)


In [561]:
# Call train_test_split
split1, split2 = 0.7,0.3
X_train, X_val, y_train, y_val = train_test_split(
    # Give set of feature and target data to split
    X_train,y_train,
    # 60% 40% split
    train_size=split1,test_size=split2,
    # Hitchhiker's + 2
    stratify=y_train,random_state=44
)

# 60k b4, now 60% in X_train, 40% in X_val
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((41475, 40), (17776, 40), (41475,), (17776,))

In [562]:
### Pipeline (without using that pipeline thing lmao) to go from:
## 1- Raw data -> numeric -> one-hot & scale features
## 2- -> subset -> encoded -> scaled

# numeric baby
X_train_numeric = X_train.select_dtypes('number')
X_val_numeric = X_val.select_dtypes('number')
# CHECKPOINT: check log regression on train
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=5000,n_jobs=-1)
model.fit(X_train_numeric,y_train)
model.score(X_val_numeric,y_val)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1544: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.



0.5508550855085509

# INVESTIGATION!

In [0]:
import plotly.express as px
import pandas_profiling
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
## HEATMAP one-sided
def heatieboi(data):
  from string import ascii_letters
  import numpy as np
  import pandas as pd
  import seaborn as sns
  import matplotlib.pyplot as plt

  sns.set(style="white")
  # Compute the correlation matrix
  corr = data.corr()

  # Generate a mask for the upper triangle
  mask = np.zeros_like(corr, dtype=np.bool)
  mask[np.triu_indices_from(mask)] = True

  # Set up the matplotlib figure
  f, ax = plt.subplots(figsize=(25, 25))

  # Generate a custom diverging colormap
  cmap = sns.diverging_palette(220, 10, as_cmap=True)

  # Draw the heatmap with the mask and correct aspect ratio
  sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
              square=True, linewidths=.5, cbar_kws={"shrink": .5})
  plt.show()
  # To assure plot closure
  plt.close()

scheme_name is 50% empty...

In [565]:
X_train.scheme_name.count()/len(X_train.index)

0.5261000602772755

In [566]:
X_train.describe(exclude='number').T.sort_values(by='unique',ascending=False).head()

,count,unique,top,freq
wpt_name,41475,27207,none,2457
subvillage,41219,16138,Madukani,371
scheme_name,21820,2441,K,485
ward,41475,2071,Igosi,231
installer,38918,1766,DWE,12184


#  Cardinality

In [0]:
### Reduce cardinality function
def reduceCard(df1,df2,col,amt):
  df1[col] = df1[col].astype(str)
  df2[col] = df2[col].astype(str)
  listoftop = df1[col].value_counts()[:amt].index
  df1.loc[~df1[col].isin(listoftop),col] = 'other'
  df2.loc[~df2[col].isin(listoftop),col] = 'other'
  return df1, df2

### Let's get and set the top wpt_name

In [568]:
### Okay so: none; Shuleni; Zahanati; Msikitini; etc
# Let's keep these
pd.value_counts(X_train.wpt_name.values,sort=True).head(20)

none               2457
Shuleni            1218
Zahanati            557
Msikitini           375
Kanisani            229
Bombani             207
Sokoni              187
Ofisini             177
School              152
Shule Ya Msingi     136
Shule               114
Sekondari           105
Muungano             92
Mkombozi             86
Madukani             80
Mbugani              67
Mkuyuni              67
Upendo               60
Kisimani             59
Center               59
dtype: int64

In [569]:
X_train, X_val = reduceCard(X_train,X_val,'wpt_name',15)
pd.value_counts(X_train.wpt_name.values,sort=True)

other              35303
none                2457
Shuleni             1218
Zahanati             557
Msikitini            375
Kanisani             229
Bombani              207
Sokoni               187
Ofisini              177
School               152
Shule Ya Msingi      136
Shule                114
Sekondari            105
Muungano              92
Mkombozi              86
Madukani              80
dtype: int64

### Get and set top ward names

In [570]:
pd.value_counts(X_train.ward.values,sort=True).head()

Igosi        231
Imalinyi     190
Siha Kati    161
Mdandu       155
Nduruma      152
dtype: int64

In [571]:
X_train, X_val = reduceCard(X_train,X_val,'ward',10)
pd.value_counts(X_train.ward.values,sort=True).head(10)

other           39891
Igosi             231
Imalinyi          190
Siha Kati         161
Mdandu            155
Nduruma           152
Kitunda           145
Mishamo           145
Maji ya Chai      140
Chalinze          135
dtype: int64

Maybe if I remove the very edges of the population data?

### Waterboiz

In [572]:
X_train.describe(exclude='number').T.sort_values(by='unique',ascending=False).head()

,count,unique,top,freq
subvillage,41219,16138,Madukani,371
scheme_name,21820,2441,K,485
installer,38918,1766,DWE,12184
funder,38941,1585,Government Of Tanzania,6392
date_recorded,41475,349,2011-03-15,407


In [573]:
X_train, X_val = reduceCard(X_train,X_val,'funder',5)
pd.value_counts(X_train.funder.values,sort=True).head(10)

other                     27844
Government Of Tanzania     6392
nan                        2534
Danida                     2174
Hesawa                     1556
Rwssp                       975
dtype: int64

In [574]:
px.scatter(X_train,x='region',y='population',color='id')

In [0]:
#heatieboi(X_train_encoded)

# CATEGORICALZ: Hard code

In [0]:
X_train.describe(exclude='number').head(0)

In [0]:
X_train.describe().head(0)
### already in the training

In [0]:
### Pipeline (without using that pipeline thing lmao) to go from:
## 1- Raw data -> numeric -> _one-hot & scale features_
## 2- -> +subset+ -> +encoded+ -> +scaled+
import category_encoders as ce
from sklearn.preprocessing import StandardScaler

# Something's wrong with this...lol I had waterpoint_type_group twice...
'''
categorical_features = ['public_meeting','permit','source_class',
                        'management_group','quantity','waterpoint_type_group',
                        'payment_type','wpt_name','ward','region']
'''
#categorical_features = ['quantity']
categorical_features = ['quantity','region','waterpoint_type_group','installer']

# Redo making the numeric features from SOURCE (X_train) so up there I can do 
# some feature engineering.

# This variable is so that we get total list of all numerical features to combo
#numeric_features = X_train.select_dtypes('number').columns.drop('id').tolist()
numeric_features = ['gps_height','longitude','region_code','district_code',
                    'population','construction_year']

features = categorical_features + numeric_features

# Make sure the training and validation sets are using the features we're doin'
X_train_subset = X_train[features]
X_val_subset = X_val[features]

encoder = ce.OneHotEncoder(use_cat_names=True)
# Encoding time!
# Always fit_transform on train set/transform on vali set, idk why
X_train_encoded = encoder.fit_transform(X_train_subset)
X_val_encoded = encoder.transform(X_val_subset)
scaler = StandardScaler()

# Scale time!
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)

### Now select best features

In [0]:
'''
from sklearn.feature_selection import f_regression, SelectKBest
### Loopy loop
for i in range(1,len(X_train_encoded.columns)+1):
  # How many to go thru
  k = 50
  # attacker
  selector = SelectKBest(score_func=f_regression,k=5)
  X_train_selected = selector.fit_transform(X_train_scaled,y_train)
  X_val_selected = selector.transform(X_val_scaled)
  
  print(f'{k} features')
  # Fit model
  model = LogisticRegression()
  model.fit(X_train_selected,y_train)
  y_pred = model.predict(X_val_selected,y_train)
  mae = mean_absolute_error(y_val,y_pred)
  r2 = model.score(X_train_selected,y_train)
  ''';

In [0]:
selector = SelectKBest(score_func=f_regression,k=1810)
X_train_selected = selector.fit_transform(X_train_scaled,y_train)
X_val_selected = selector.transform(X_val_scaled)

# Fit model!
model = LogisticRegression()
model.fit(X_train_selected,y_train)

# Here we use the validation datasets to test
print("Validation Accuracy",model.score(X_val_selected,y_val))

### Kaggle krap

In [0]:
X_test_subset = test_features[features]
X_test_encoded = encoder.transform(X_test_subset)
X_test_scaled = scaler.transform(X_test_encoded)
all(X_test_encoded.columns == X_train_encoded.columns)

In [0]:
y_pred = model.predict(X_test_scaled)
y_pred.shape

submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('sub-01.csv',index=False)

if in_colab:
    from google.colab import files
    # Just try again if you get this error:
    # TypeError: Failed to fetch
    # https://github.com/googlecolab/colabtools/issues/337
    files.download('sub-01.csv')

In [0]:
y_pred.shape